In [1]:
import plotnine as p9
import polars as pl
from p9customtheme import custom_discrete, custom_theme

# Download a example dataset from OAS
url1 = "https://opig.stats.ox.ac.uk/webapps/ngsdb/paired/Alsoiussi_2020/csv/SRR11528761_paired.csv.gz"
df1 = pl.read_csv(url1, has_header=True, skip_rows=1, separator=",")


url2 = "https://opig.stats.ox.ac.uk/webapps/ngsdb/paired/Alsoiussi_2020/csv/SRR11528762_paired.csv.gz"
df2 = pl.read_csv(url2, has_header=True, skip_rows=1, separator=",")


In [2]:
from cstbioinfo.immune import ruzicka_similarity
df1 = df1.with_columns(pl.lit("SRR11528761").alias("sample_id"))
df2 = df2.with_columns(pl.lit("SRR11528762").alias("sample_id"))
df_combined = pl.concat([df1, df2], how="vertical")

print(f"Combined DataFrame shape: {df_combined.shape}")

Combined DataFrame shape: (11388, 181)


In [12]:
df_count = (df_combined
                .group_by(["sample_id", "v_call_heavy", "j_call_heavy", "junction_aa_heavy"])
                .agg(pl.len().alias("count"))
            )
df_count.head(2)

sample_id,v_call_heavy,j_call_heavy,junction_aa_heavy,count
str,str,str,str,u32
"""SRR11528762""","""IGHV14-4*01""","""IGHJ4*01""","""CTTGSYGNYVNSMDYW""",1
"""SRR11528761""","""IGHV2-2*03""","""IGHJ2*01""","""CASPIYYEGFDYW""",1


In [17]:
ruzicka_similarity(
    df_count,
    feature_columns=["v_call_heavy", "j_call_heavy", "junction_aa_heavy"],
    count_column="count",
    sample_column="sample_id",
)

Numerator: 5412.0, Denominator: 5976.0


sample_1,sample_2,similarity
str,str,f64
"""SRR11528761""","""SRR11528762""",0.905622
